In [142]:
# initialize from http://api.steampowered.com/ISteamApps/GetAppList/v0001/
import requests
import json
import sys
from lxml import etree

request = 'http://api.steampowered.com/ISteamApps/GetAppList/v0001/'
response = requests.get(request)
if len(response.text)<1000:
    sys.exit("something wrong with request")
json_response = json.loads(response.text)
root = etree.Element("root")
for r_app in json_response['applist']['apps']['app']:
    app = etree.SubElement(root, 'app')
    app.set('id',str(r_app['appid']))
    name = etree.SubElement(app, 'name')
    name.text = str(r_app['name'])
tree = etree.ElementTree(root)
with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_working.xml', 'wb') as f:
    f.write(etree.tostring(tree, pretty_print=True, xml_declaration=True))

In [146]:
# append data from _appids_scrapped files:
# it's a file of api responses
# requests = appids found in users libraries

import json
import datetime
from lxml import etree
from io import BytesIO

with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_working.xml', 'rb') as f:
    xml = f.read()
    tree = etree.parse(BytesIO(xml))
    
with open("C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_appids_scrapped.json", 'r') as f:
    json_data = json.loads(f.read())

for app in json_data:
    requested_app = None
    
    for xml_app in tree.iter('app'):
        if xml_app.get('id') == app['appid']:
            requested_app = xml_app
            break
    if requested_app == None:
        print(app, 'not in xml')
        continue
    
    requested_app.set('scraped', '1')
    
    for key in list(app.keys()):
        if key == 'requested_appid': # id as it was requested by scraper, not as was given in response
            if app['requested_appid'] != app['appid']:
                query = requested_app.xpath(".//redirected_from")
                if len(query) == 0:
                    sub = etree.SubElement(requested_app, 'redirected_from')
                else:
                    sub = query[0]
                red_id = etree.SubElement(sub, 'redirected_from')
                red_id.text = app['requested_appid']
                
                # then flag the original:
                app_to_flag = None
                for xml_app2 in tree.iter('app'):
                    if xml_app2.get('id') == app['requested_appid']:
                        app_to_flag = xml_app2
                        break
                if app_to_flag == None:
                    print(app['requested_appid'], 'not in xml')
                    continue
                app_to_flag.set('redirects_to_id',app['requested_appid'])
        
        elif key == 'title': # title as scraped
            #name from json?
            if requested_app.xpath(".//name")[0].text != app['title']:
                query = requested_app.xpath(".//name_scraped")
                if len(query) == 0:
                    sub = etree.SubElement(requested_app, 'name_scraped')
                else:
                    sub = query[0]
                sub.text = str(app['title'])
                #print("names don't match! found in xml:",requested_app.xpath(".//name")[0].text,'found in json:', app['title'])

        elif key == 'release_date': # it's a list
            query = requested_app.xpath(".//release_date")
            if len(query) == 0:
                sub = etree.SubElement(requested_app, 'release_date')
            else:
                sub = query[0]
            try:
                date = app['release_date'][0]
                # let's reformat date to YYYY-MM-DD format
                try:
                    date_formatted = datetime.datetime.strptime(date, "%d %b, %Y").strftime("%Y-%m-%d")
                except ValueError:
                    try:
                        date_formatted = datetime.datetime.strptime(date, "%b %Y").strftime("%Y-%m-%d")
                    except ValueError: #wrong date format
                        date_formatted = None
            except IndexError: #sometimes there's no date
                date_formatted = None
            sub.text = date_formatted
                
        elif key == 'price': # price in EUR, it's a list
            query = requested_app.xpath(".//price")
            if len(query) == 0:
                sub = etree.SubElement(requested_app, 'price')
            else:
                sub = query[0]
            try:
                price = app['price'][0]
            except IndexError: # sometimes there's no price
                price = None
            sub.text = price
            if price != None:
                price = price.replace(',','.')
                if price != '0.00':
                    sub.set('currency', app['price_currency'][0])
            
        elif key == 'developer': # may contain several developers, it's the same for and publishers and tags
            query = requested_app.xpath(".//developers")
            if len(query) != 0:
                sub = query[0]
                requested_app.remove(sub)
            sub = etree.SubElement(requested_app, 'developers')
            for dev_name in app['developer']:
                dev_name = dev_name.title()
                dev_sub = etree.SubElement(sub, 'developer')
                dev_sub.text = dev_name
                
        elif key == 'publisher':
            query = requested_app.xpath(".//publishers")
            if len(query) != 0:
                sub = query[0]
                requested_app.remove(sub)
            sub = etree.SubElement(requested_app, 'publishers')
            for pub_name in app['publisher']:
                pub_name = pub_name.title()
                pub_sub = etree.SubElement(sub, 'publisher')
                pub_sub.text = pub_name
                
        elif key == 'tags':
            query = requested_app.xpath(".//tags")
            if len(query) != 0:
                sub = query[0]
                requested_app.remove(sub)
            sub = etree.SubElement(requested_app, 'tags')
            for tag_name in app['tags']:
                tag_name = tag_name.title()
                tag_sub = etree.SubElement(sub, 'tag')
                tag_sub.text = tag_name
                
with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_working.xml', 'wb') as f:
    f.write(etree.tostring(tree, pretty_print=True, xml_declaration=True))

{'appid': '', 'title': None, 'release_date': [], 'price': [], 'price_currency': [], 'developer': [], 'publisher': [], 'tags': []} not in xml
{'appid': '901147', 'title': 'Earthworm Jim', 'release_date': ['30 Nov, 1995'], 'price': ['19,99'], 'price_currency': ['EUR'], 'developer': ['Shiny Entertainment'], 'publisher': ['Interplay Inc.'], 'tags': ['Action', 'Adventure', 'Platformer', 'Classic', '2D']} not in xml
{'appid': '901638', 'title': 'Dungeon Siege III', 'release_date': ['16 Jun, 2011'], 'price': ['14,99'], 'price_currency': ['EUR'], 'developer': ['Obsidian Entertainment'], 'publisher': ['SQUARE ENIX, Eidos Interactive'], 'tags': ['RPG', 'Action', 'Fantasy', 'Action RPG', 'Co-op', 'Local Co-Op', 'Singleplayer', 'Dungeon Crawler', 'Female Protagonist']} not in xml
{'appid': '39391', 'title': 'Freeze Tag Fun Pack #1', 'release_date': ['30 Nov, 1999'], 'price': ['18,99'], 'price_currency': ['EUR'], 'developer': ['Freeze Tag Inc.', 'Joju Games', 'Linksolutions Ltd.', 'Dekovir Entertai

In [147]:
# append data from scraped_meta files:
# these are meta files generated when scraping users
# they contain values regarding number of users playing specific game etc

import json
from lxml import etree
from io import BytesIO

with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_working.xml', 'rb') as f:
    xml = f.read()
    tree = etree.parse(BytesIO(xml))
    
with open("C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\profiles0_scraped_meta.txt", 'r') as f:
    json_data = json.loads(f.read())
    
for app in json_data:
    
    requested_app = None
    
    # does this app exist in xml?
    # query = tree.xpath("//app[@id="+app+"]")  is apparently slower
    for xml_app in tree.iter('app'):
        if xml_app.get('id') == app:
            requested_app = xml_app
            break
    if requested_app == None:
        print(app,'not in xml')
        continue
    
    for key in list(json_data[app].keys()):
        if key == 'number': # number of players
            query = requested_app.xpath(".//number_of_players")
            if len(query) == 0:
                sub = etree.SubElement(requested_app, 'number_of_players')
                sub.text = str(json_data[app]['number'])
            else:
                sub = query[0]
                sub.text = str(int(sub.text) + json_data[app]['number'])
        elif key == 'user': # last user from which the data was collected
            query = requested_app.xpath(".//found_in_user")
            if len(query) == 0:
                sub = etree.SubElement(requested_app, 'found_in_user')
            else:
                sub = query[0]
            sub.text = str(json_data[app]['user'])
        elif key == 'total_playtime': # total playtime of all users
            query = requested_app.xpath(".//total_playtime")
            if len(query) == 0:
                sub = etree.SubElement(requested_app, 'total_playtime')
                sub.text = str(json_data[app]['total_playtime'])
            else:
                sub = query[0]
                sub.text = str(int(sub.text) + json_data[app]['total_playtime'])

with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_working.xml', 'wb') as f:
    f.write(etree.tostring(tree, pretty_print=True, xml_declaration=True))

In [3]:
# color from PCA to color generation

from lxml import etree
from io import BytesIO
from colormath.color_objects import sRGBColor # https://anaconda.org/melund/colormath

with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_working.xml', 'rb') as f:
    xml = f.read()
    tree = etree.parse(BytesIO(xml))
    
with open("C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_colors_from_pca.csv", 'r') as f:
    for line in f.readlines():
        if len(line)<10:
            continue
        line = line.strip("\n") # appid10170,0.0,0.0196078431372549,0.4
        split = line.split(',')
        if len(split)!=4:
            print('line seems errorenous:',line)
            continue
        app = split[0]
        app = app.strip('appid')
        color_from_pca = sRGBColor(split[1],split[2],split[3]).get_rgb_hex()
        #color_from_pca = colour.rgb2hex((split[1],split[2],split[3]))
        
        #find app
        requested_app = None
        for xml_app in tree.iter('app'):
            if xml_app.get('id') == app:
                requested_app = xml_app
                break
        if requested_app == None:
            print(app,'not in xml')
            continue
            
        query = requested_app.xpath(".//colors")
        if len(query) == 0:
            sub = etree.SubElement(requested_app, 'colors')
        else:
            sub = query[0]
        query = sub.xpath(".//color[@type='pca']")
        if len(query) == 0:
            color = etree.SubElement(sub, 'color')
            color.set('type','pca')
        else:
            color = query[0]
        color.text = color_from_pca

with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_working.xml', 'wb') as f:
    f.write(etree.tostring(tree, pretty_print=True, xml_declaration=True))

In [26]:
# create a 'standardized size' field which takes in number of players and distributes it between 0 and 1
# in such a way that they're equally distributed

from lxml import etree
from io import BytesIO
import numpy as np
import math

with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_working.xml', 'rb') as f:
    xml = f.read()
    tree = etree.parse(BytesIO(xml))

numbers = []

for sub in tree.iter('number_of_players'):
    numbers.append(math.log(int(sub.text)))
amin = np.amin(numbers)
amax = np.amax(numbers)

for app in tree.iter('app'):
    query = app.xpath(".//number_of_players")
    if len(query) == 0:
        continue
    else:
        sub = query[0]
    standardized_size = math.log(int(sub.text))
    standardized_size = (standardized_size - amin)/(amax - amin)
    print(sub.text, standardized_size)
    
    query = app.xpath(".//node_sizes")
    if len(query) == 0:
        sub = etree.SubElement(app, 'node_sizes')
    else:
        sub = query[0]
    query = app.xpath(".//size[@type='std_number_of_players']")
    if len(query) == 0:
        sub2 = etree.SubElement(sub, 'size')
    else:
        sub2 = query[0]
    sub2.set('type','std_number_of_players')
    sub2.text = str(standardized_size)

with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_working.xml', 'wb') as f:
    f.write(etree.tostring(tree, pretty_print=True, xml_declaration=True))

8950 0.899623995599
4099 0.822418340761
3751 0.813646874866
3662 0.811272798285
4219 0.825271134097
3666 0.811380730831
4700 0.835945161932
8379 0.893106245474
8379 0.893106245474
4157 0.823807472082
8495 0.89446557464
10905 0.919156780412
3347 0.802380275312
4092 0.822249359297
8666 0.896435935368
9814 0.908735149449
6718 0.871262936043
6629 0.869944404169
8918 0.899269874055
6556 0.868849628812
5250 0.846886246394
18733 0.972649849524
11627 0.925494952278
24703 1.0
30 0.336263469614
1245 0.704608631031
1245 0.704608631031
6882 0.873647471811
1245 0.704608631031
124 0.476562934131
124 0.476562934131
124 0.476562934131
5 0.159119014815
1250 0.705004888886
1364 0.713633734123
1527 0.72479409383
1225 0.703007523931
85 0.439228059461
182 0.514500379756
431 0.599733071779
469 0.608086739771
153 0.497340294992
204 0.525782329068
312 0.567788917029
647 0.639896300422
6 0.177144454798
2770 0.783673098956
1619 0.730578127395
1619 0.730578127395
490 0.61241733874
368 0.584109725082
485 0.611403

In [28]:
# dump all ids into one file (for scraping purposes)
from lxml import etree
from io import BytesIO

with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_working.xml', 'rb') as f:
    xml = f.read()
    tree = etree.parse(BytesIO(xml))

with open('C:\\Users\\Admin\\Documents\\GitHub\\GamesGraph\\scripts\\wip_data\\_appids_from_xml.txt', 'w') as f:
    for app in tree.iter('app'):
        appid = app.get('id')
        f.write(appid+'\n')
    